In [1]:
# 以 CIFAR10 数据集为例，展示一下完整的模型训练套路，完成对数据集的分类问题

import torch
import torchvision

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

# 准备数据集
train_data = torchvision.datasets.CIFAR10(root="dataset", train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="dataset", train=False, transform=torchvision.transforms.ToTensor(), download=True)

# 获得数据集的长度 len(), 即length
train_data_size = len(train_data)
test_data_size = len(test_data)

# 格式化字符串, format() 中的数据会替换 {}
print("训练数据集及的长度为: {}".format(train_data_size))
print("测试数据集及的长度为: {}".format(test_data_size))

# 利用DataLoader 来加载数据
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型
class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )


    def forward(self, input):
        input = self.model(input)
        return input

model = Model()
if torch.cuda.is_available():
    model = model.cuda()                        # 在 GPU 上进行训练

# 创建损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()                    # 在 GPU 上进行训练

# 优化器
learning_rate = 1e-2        # 1e-2 = 1 * (10)^(-2) = 1 / 100 = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 设置训练网络的一些参数
total_train_step = 0                        # 记录训练的次数
total_test_step = 0                         # 记录测试的次数
epoch = 10                                  # 训练的轮数

# 添加tensorboard
writer = SummaryWriter("logs_train")
start_time = time.time()                    # 开始训练的时间
for i in range(epoch):
    print("------第 {} 轮训练开始------".format(i+1))

    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
        targets = targets.cuda()            # 在gpu上训练
        outputs = model(imgs)               # 将训练的数据放入
        loss = loss_fn(outputs, targets)    # 得到损失值

        optimizer.zero_grad()               # 优化过程中首先要使用优化器进行梯度清零
        loss.backward()                     # 调用得到的损失，利用反向传播，得到每一个参数节点的梯度
        optimizer.step()                    # 对参数进行优化
        total_train_step += 1               # 上面就是进行了一次训练，训练次数 +1

        # 只有训练步骤是100 倍数的时候才打印数据，可以减少一些没有用的数据，方便我们找到其他数据
        if total_train_step % 100 == 0:
            end_time = time.time()          # 训练结束时间
            print("训练时间: {}".format(end_time - start_time))
            print("训练次数: {}, Loss: {}".format(total_train_step, loss))
            writer.add_scalar("train_loss", loss.item(), total_train_step)


    # 如何知道模型有没有训练好，即有咩有达到自己想要的需求
    # 我们可以在每次训练完一轮后，进行一次测试，在测试数据集上跑一遍，以测试数据集上的损失或正确率评估我们的模型有没有训练好

    # 顾名思义，下面的代码没有梯度，即我们不会利用进行调优
    total_test_loss = 0
    total_accuracy = 0                                      # 准确率
    with torch.no_grad():
        for data in test_dataloader:                        # 测试数据集中取数据
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()                          # 在 GPU 上进行训练
                targets = targets.cuda()
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)                # 这里的 loss 只是一部分数据(data) 在网络模型上的损失
            total_test_loss = total_test_loss + loss        # 整个测试集的loss
            accuracy = (outputs.argmax(1) == targets).sum() # 分类正确个数
            total_accuracy += accuracy                      # 相加

    print("整体测试集上的loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy / test_data_size))
    writer.add_scalar("test_loss", total_test_loss)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    total_test_loss += 1                                    # 测试完了之后要 +1

    torch.save(model, "model_{}.pth".format(i))
    print("模型已保存")

writer.close()

Files already downloaded and verified
Files already downloaded and verified
训练数据集及的长度为: 50000
测试数据集及的长度为: 10000
------第 1 轮训练开始------
训练时间: 3.8432178497314453
训练次数: 100, Loss: 2.301461696624756
训练时间: 4.9580583572387695
训练次数: 200, Loss: 2.2890772819519043
训练时间: 6.052404403686523
训练次数: 300, Loss: 2.265805721282959
训练时间: 7.069671630859375
训练次数: 400, Loss: 2.210125684738159
训练时间: 8.067373514175415
训练次数: 500, Loss: 2.1316375732421875
训练时间: 9.118994951248169
训练次数: 600, Loss: 2.0457425117492676
训练时间: 10.191678524017334
训练次数: 700, Loss: 2.000765085220337
整体测试集上的loss: 315.400390625
整体测试集上的正确率: 0.28450000286102295
模型已保存
------第 2 轮训练开始------
训练时间: 12.443027019500732
训练次数: 800, Loss: 1.846737027168274
训练时间: 13.443705558776855
训练次数: 900, Loss: 1.8030729293823242
训练时间: 14.469829320907593
训练次数: 1000, Loss: 1.8910515308380127
训练时间: 15.478224277496338
训练次数: 1100, Loss: 2.0446183681488037
训练时间: 16.47923755645752
训练次数: 1200, Loss: 1.695770263671875
训练时间: 17.4947566986084
训练次数: 1300, Loss: 1.629657983779

In [2]:
# 以 CIFAR10 数据集为例，展示一下完整的模型训练套路，完成对数据集的分类问题

import torch
import torchvision

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import time

# 定义训练的设备
device = torch.device("cuda")

# 准备数据集
train_data = torchvision.datasets.CIFAR10(root="dataset", train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="dataset", train=False, transform=torchvision.transforms.ToTensor(), download=True)

# 获得数据集的长度 len(), 即length
train_data_size = len(train_data)
test_data_size = len(test_data)

# 格式化字符串, format() 中的数据会替换 {}
print("训练数据集及的长度为: {}".format(train_data_size))
print("测试数据集及的长度为: {}".format(test_data_size))

# 利用DataLoader 来加载数据
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型
class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )


    def forward(self, input):
        input = self.model(input)
        return input

model = Model()
model = model.to(device)                    # 在 GPU 上进行训练

# 创建损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)                # 在 GPU 上进行训练

# 优化器
learning_rate = 1e-2        # 1e-2 = 1 * (10)^(-2) = 1 / 100 = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 设置训练网络的一些参数
total_train_step = 0                        # 记录训练的次数
total_test_step = 0                         # 记录测试的次数
epoch = 10                                  # 训练的轮数

# 添加tensorboard
writer = SummaryWriter("logs_train")
start_time = time.time()                    # 开始训练的时间
for i in range(epoch):
    print("------第 {} 轮训练开始------".format(i+1))

    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)               # 将训练的数据放入
        loss = loss_fn(outputs, targets)    # 得到损失值

        optimizer.zero_grad()               # 优化过程中首先要使用优化器进行梯度清零
        loss.backward()                     # 调用得到的损失，利用反向传播，得到每一个参数节点的梯度
        optimizer.step()                    # 对参数进行优化
        total_train_step += 1               # 上面就是进行了一次训练，训练次数 +1

        # 只有训练步骤是100 倍数的时候才打印数据，可以减少一些没有用的数据，方便我们找到其他数据
        if total_train_step % 100 == 0:
            end_time = time.time()          # 训练结束时间
            print("训练时间: {}".format(end_time - start_time))
            print("训练次数: {}, Loss: {}".format(total_train_step, loss))
            writer.add_scalar("train_loss", loss.item(), total_train_step)


    # 如何知道模型有没有训练好，即有咩有达到自己想要的需求
    # 我们可以在每次训练完一轮后，进行一次测试，在测试数据集上跑一遍，以测试数据集上的损失或正确率评估我们的模型有没有训练好

    # 顾名思义，下面的代码没有梯度，即我们不会利用进行调优
    total_test_loss = 0
    total_accuracy = 0                                      # 准确率
    with torch.no_grad():
        for data in test_dataloader:                        # 测试数据集中取数据
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)                # 这里的 loss 只是一部分数据(data) 在网络模型上的损失
            total_test_loss = total_test_loss + loss        # 整个测试集的loss
            accuracy = (outputs.argmax(1) == targets).sum() # 分类正确个数
            total_accuracy += accuracy                      # 相加

    print("整体测试集上的loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy / test_data_size))
    writer.add_scalar("test_loss", total_test_loss)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    total_test_loss += 1                                    # 测试完了之后要 +1

    torch.save(model, "model_{}.pth".format(i))
    print("模型已保存")

writer.close()


Files already downloaded and verified
Files already downloaded and verified
训练数据集及的长度为: 50000
测试数据集及的长度为: 10000
------第 1 轮训练开始------
训练时间: 1.2342050075531006
训练次数: 100, Loss: 2.282177448272705
训练时间: 2.233593225479126
训练次数: 200, Loss: 2.2736942768096924
训练时间: 3.2655892372131348
训练次数: 300, Loss: 2.204333543777466
训练时间: 4.284208297729492
训练次数: 400, Loss: 2.1174211502075195
训练时间: 5.312605857849121
训练次数: 500, Loss: 1.9917079210281372
训练时间: 6.333043336868286
训练次数: 600, Loss: 2.0082876682281494
训练时间: 7.33498477935791
训练次数: 700, Loss: 1.9690537452697754
整体测试集上的loss: 304.3615417480469
整体测试集上的正确率: 0.3132999837398529
模型已保存
------第 2 轮训练开始------
训练时间: 9.588040351867676
训练次数: 800, Loss: 1.800155520439148
训练时间: 10.646360874176025
训练次数: 900, Loss: 1.7634451389312744
训练时间: 11.692577838897705
训练次数: 1000, Loss: 1.868146300315857
训练时间: 12.71735143661499
训练次数: 1100, Loss: 1.9585095643997192
训练时间: 13.73293423652649
训练次数: 1200, Loss: 1.6727432012557983
训练时间: 14.749567747116089
训练次数: 1300, Loss: 1.601604700